In [67]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 10000:
            i = i +1
            X.append(row)
    #for row in reader:
        #data.append(row)
        #print(row)
y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 10000:
            j = j +1
            y.append(row.replace("\n",""))


In [68]:
# Faire plus propre en créant directement un tableau numpy TODO
X = np.array(X)
y = np.array(y)

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [71]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial',
                        n_jobs=-1, max_iter= 1000)

In [ ]:
%time clf.fit(X_dtm, y)

In [51]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))
        #print(row)
test = np.asarray(test)
test_dtm = vect.transform(test)
clf.predict(test_dtm)

array(['0', '2', '0', ..., '0', '0', '2'], dtype='<U2')

In [44]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

print (test_label.shape)
print (test.shape)
clf.score(test_dtm, test_label)

(50000, 1)
(50000,)


0.20982

In [65]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial',
                        n_jobs=-1, max_iter= 1000),
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    model.predict(test_dtm)
    score = model.score(test_dtm, test_label)
    #accuracies = cross_val_score(model, test_dtm, test_label, scoring='accuracy', cv=CV)
    #for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, score))
cv_df = pd.DataFrame(entries, columns=['model_name', 'score'])


In [66]:
print (cv_df)

               model_name    score
0      LogisticRegression  0.20982
1  RandomForestClassifier  0.21596
2               LinearSVC  0.17370
3           MultinomialNB  0.22012
